In [1]:
# Code repo
# https://github.com/whatifif/handgesturecode


In [2]:
import cv2
import numpy as np
import copy
import math
from appscript import app
import uuid
import os
import test64x64 as ML

# Environment:
# OS    : Mac OS High Siera
# python: 2.7.13
# opencv: 3.3.0

# Values to be initialised
Inited = False
frameW = None # frame width
frameH = None # frame height
marginX = 10 # margin between region
marginY = 10
gapX = 2 # margin between buttons
gapY = 2

# parameters
cap_region_x_end=0.5 # start point/total width
cap_region_y_end=0.8  # end point/total heighty
capStartX = 0 # keyboard region start x
capStartY = 0 # keyboard region start y
capEndX = None # keyboard region end x
capEndY = None # keyboard region end y


# buttons for capture 1
btnsX = None # btns start x 
btnsY = None # btns start y
btnsW = 600 # btns width
btnsH = 120 # btns height
bW = btnsW / 10  # btn width
bH = btnsH / 3  # btn height
bColor = (0, 255, 255) 
bThick = 2
bTitles = [
            ['lc', 'lo', 'lq', 'lw', 'le', 'l1', 'l2', 'l3', 'l4', 'l5' ],
            ['mc', 'mo', 'mq', 'mw', 'me', 'm1', 'm2', 'm3', 'm4', 'm5' ],
            ['rc', 'ro', 'rq', 'rw', 're', 'r1', 'r2', 'r3', 'r4', 'r5' ]
          ]
btns = [[(bW * j, bH * i, bTitles[i][j] ) for j in range(10)] for i in range(3)] 

# buttons for capture 2
btns2X = None # btns2 start x
btns2Y = None # btns2 start y
btns2W = 300 # btns2 width
btns2H = 120 # btns2 height
bW2 = btns2W / 5  # btn width
bH2 = btns2H / 2  # btn height
bTitles2 = [
            ['vc', 'vo', 'vq', 'vw', 've'],
            ['v1', 'v2', 'v3', 'v4', 'v5']
           ]
btns2 = [[(bW2 * j, bH2 * i, bTitles2[i][j] ) for j in range(5)] for i in range(2)] 

# Mouse Mode
MouseMode = False
mouse_region_x_begin = 0.7
mouse_region_y_end = 0.8
capStartX2 = None # mouse region start x
capStartY2 = 0 # mouse region start y
capEndX2 = None # mouse region end x
capEndY2 = None # mouse region end y

# GUI Mode
GUIMode = False
GUIInited = False

# Demo Mode
DemoMode = False

# variables for getSerialNumber
uid = str(uuid.uuid4())[:8]
inited = False
currentNum = 0
def getSerialNumber(serialPrefix = 'a', startNum = 0, endNum = 90000):
    global uid, inited, currentNum
    
    if not inited:
        currentNum = startNum
        inited = True
    else:
        currentNum += 1
        
    if currentNum > endNum:
        print ("!!! Exceeding Maximum Number !!!")
    
    return serialPrefix + '_' + uid + '_' + str(currentNum)
    
    
def printThreshold(thr):
    print("! Changed threshold to "+str(thr))


# GUI mode to capture a hand
ix, iy = -1, -1
currentClass = None
detectedClass = None
currentClass2 = None
detectedClass2 = None
imgClass = None
redColor = (0,0,255)

def handleClick(event,x,y,flags,param):
    global ix, iy, GUIMode
    
    if GUIMode and event == cv2.EVENT_LBUTTONDOWN:
        ix,iy = x,y
        detectBtn(ix, iy)

def detectBtn(ix, iy):
    global btnsX, btnsY, btnsW, btnsH, btns, btns2X, btns2Y, btns2W, btns2H, btns2
    global currentClass, detectedClass, currentClass2, detectedClass2
    
    if ((btnsX < ix) and (ix < btnsX + btnsW) and (btnsY < iy) and (iy < btnsY + btnsH)):
        nX = (ix - btnsX) // bW
        nY = (iy - btnsY) // bH
        currentClass = detectedClass = btns[nY][nX][2]
    
    elif MouseMode and ( btns2X < ix) and (ix < btns2X + btns2W) and (btns2Y < iy) and (iy < btns2Y + btns2H):
        nX = (ix - btns2X) // bW2
        nY = (iy - btns2Y) // bH2
        currentClass2 = detectedClass2 = btns2[nY][nX][2] 
    
def drawBtns(frame, btns, btnsX, btnsY, bW, bH, Class):
    numRows = len(btns)
    numCols = len(btns[0])
    
    for i in range(numRows):
        for j in range(numCols):
            drawBtn(frame, btns[i][j], btnsX, btnsY, bW, bH, Class)
            
def drawBtn(frame, btn, btnsX, btnsY, bW, bH, Class):
    global bColor, bThick, currentClass2, btns2X, btns2Y, bW2, bH2, redColor, gapX, gapY

    startPoint = (btnsX + btn[0] + gapX, btnsY + btn[1] + gapY)
    endPoint = (btnsX + btn[0] + bW - gapX, btnsY + btn[1] + bH - gapY)
    bTitle = btn[2]

    if Class == bTitle:
        color = redColor
    else:
        color = bColor

    cv2.rectangle(frame, startPoint, endPoint, color, bThick)
    cv2.putText(frame, bTitle, (startPoint[0]+8, startPoint[1]+32), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

# file path for cropped image
outDirPath = os.path.join('fdata', 'pic')
csvFilePath = os.path.join('fdata', 'fdata.csv')
csvFile = None

# margin for cropped image
margin = 10             
def getCrop(x, y, w, h, margin, img):
    # make a crop a square to prevent a shape destortion 
    if w > h: 
        delta = int((w-h)/2)

        y1 = y - delta - margin
        if y1 < 0: y1 = y

        y2 = y + h + delta + margin
        if y2 > img.shape[0]: y2 = y + h

        x1 = x - margin
        if x1 < 0: x1 = x

        x2 = x + w + margin
        if x2 > img.shape[1]: x2 = x + w

        crop = img[y1 : y2, x1 : x2]
    else:
        delta = int((h-w)/2)

        y1 = y - margin
        if y1 < 0: y1 = y

        y2 = y + h + margin
        if y2 > img.shape[0]: y2 = y + h

        x1 = x - delta - margin
        if x1 < 0: x1 = x

        x2 = x + w + delta + margin
        if x2 > img.shape[1]: x2 = x + w

        crop = img[y1 : y2, x1 : x2]
    return crop

def saveCrop(crop, imgClass):
    global outDirPath
    
    # make a crop a 200x200 pixel
    f_im = cv2.resize(crop,(200, 200), interpolation=cv2.INTER_CUBIC)

    # save into image file
    fileName = getSerialNumber('a', 0, 90000) + '_' + imgClass + '.jpg'
    out_path = os.path.join(outDirPath, fileName)
    cv2.imwrite(out_path, f_im)
    print out_path
    
    return fileName

def saveCsv(fileName, imgClass):
    global csvFilePath
    
    # save into csv file
    csvFile = open(csvFilePath, 'a') # to append a data to an existing csv file
    csvFile.write(fileName + ', ' + imgClass + '\r\n')
    csvFile.close()
    
def initParams(frame):
    global cap_region_x_end, cap_region_y_end, capStartX, capStartY, capEndX, capEndY
    global MouseMode, mouse_region_x_begin, mouse_region_y_end, capStartX2, capStartY2, capEndX2, capEndY2
    global frameW, frameH, btnsX, btnsY, btns2X, btns2Y, marginX, marginY
    
    frameW = frame.shape[1]
    frameH = frame.shape[0]
    capStartX = 0
    capStartY = 0
    capEndX = int(cap_region_x_end * frameW)
    capEndY = int(cap_region_y_end * frameH)
    capStartX2 = int(mouse_region_x_begin * frameW)
    capStartY2 = 0
    capEndX2 = frameW
    capEndY2 = int(mouse_region_y_end * frameH)
    
    btnsX = marginX
    btnsY = capEndY + marginY
    btns2X = capStartX2 + marginX
    btns2Y = frameH - btns2H - marginY
    
# track hand by skin color
skin_ycrcb_mint = np.array((0, 133, 77))
skin_ycrcb_maxt = np.array((255, 173, 127))

def trackHand(img):
    global skin_ycrcb_mint, skin_ycrcb_maxt
    
    im_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
    skin_ycrcb = cv2.inRange(im_ycrcb, skin_ycrcb_mint, skin_ycrcb_maxt)    
    _, contours, _ = cv2.findContours(skin_ycrcb, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    ## get the coutours
    length = len(contours)
    maxArea = -1
    res = None
    if length > 0:
        for i in range(length):  # find the biggest contour (according to area)
            temp = contours[i]
            area = cv2.contourArea(temp)
            if area > maxArea:
                maxArea = area
                ci = i

        # draw the bounding rect
        res = contours[ci]
    return res

def drawGuide(frame, btnsX, btnsY, btnsW, marginX):
    cv2.putText(frame,"d: Demo mode", 
                (btnsX + btnsW + marginX, btnsY + 25), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,(0,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,"g: GUI mode", 
                (btnsX + btnsW + marginX, btnsY + 55), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,(0,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,"m: Mouse mode.", 
                (btnsX + btnsW + marginX, btnsY + 85), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,(0,255,255),2,cv2.LINE_AA)
    cv2.putText(frame,"esc: quit", 
                (btnsX + btnsW + marginX, btnsY + 115),  
                cv2.FONT_HERSHEY_SIMPLEX, 
                1,(0,255,255),2,cv2.LINE_AA) 

# Camera
camera = cv2.VideoCapture(0)
camera.set(10,200)

cv2.namedWindow('main')
cv2.setMouseCallback('main', handleClick)

# main loop
prevClass = None

while camera.isOpened():
    ret, frame = camera.read()
    threshold = cv2.getTrackbarPos('trh1', 'trackbar')
    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # smoothing filter
    frame = cv2.flip(frame, 1)  # flip the frame horizontally
    
    # init
    if not Inited:
        initParams(frame)
        Inited = True

    # keyboard region
    cv2.rectangle(frame, (capStartX, capStartY), (capEndX, capEndY), (255, 0, 0), 2)
    
    # mouse region
    if MouseMode:
        cv2.rectangle(frame, (capStartX2, capStartY2), (capEndX2, capEndY2), (255, 0, 0), 2)
        
    if GUIMode:
        if not GUIInited:
            if not os.path.exists(outDirPath):
                os.makedirs(outDirPath)
            GUIInited = True
            
        drawBtns(frame, btns, btnsX, btnsY, bW, bH, currentClass)
        
        if MouseMode:
            drawBtns(frame, btns2, btns2X, btns2Y, bW2, bH2, currentClass2)
        
    #  Main operation
    if True:
        img = frame[capStartY : capEndY, capStartX : capEndX]  # clip the ROI
        img = copy.deepcopy(img)
        res = trackHand(img)
        (x,y,w,h) = cv2.boundingRect(res)
        cv2.rectangle(frame, 
                      (capStartX + x - margin, y - margin), 
                      (capStartX + x + w + margin, y + h + margin), 
                      (0, 255, 255), 2)    
        
        # mouse
        if MouseMode:
            img2 = frame[capStartY2 : capEndY2, capStartX2: capEndX2]  # clip the ROI
            img2 = copy.deepcopy(img2)
            res2 = trackHand(img2)
            (x2,y2,w2,h2) = cv2.boundingRect(res2)
            cv2.rectangle(frame, 
                          (capStartX2 + x2 - margin, y2 - margin), 
                          (capStartX2 + x2 + w2 + margin, y2 + h2 + margin), 
                          (255, 0, 255), 2)
            
            #Find moments of the largest contour
            moments2 = cv2.moments(res2)

            #Central mass of first order moments
            if moments2['m00']!=0:
                cx2 = int(moments2['m10']/moments2['m00']) # cx = M10/M00
                cy2 = int(moments2['m01']/moments2['m00']) # cy = M01/M00
                centerMass2=(int(mouse_region_x_begin * frame.shape[1]) + cx2,cy2)    

                #Draw center mass
                cv2.circle(frame,centerMass2,5,[100,0,255],2)

        drawGuide(frame, btnsX, btnsY, btnsW, marginX)
           
        cv2.imshow('main', frame)

    # Keyboard OP
    k = cv2.waitKey(10)
    if k == 27:  # press ESC to exit
        if csvFile and (not csvFile.closed):
            csvFile.close()
        break
        
    elif k == ord('g'): # GUI mode to capture a hand
        if not GUIMode: 
            GUIMode = True
            print '!!! GUI Mode on !!!'
        else:
            GUIMode = False
            print '!!! GUI Mode off !!!'
    
    elif k == ord('m'): # Mouse mode to capture a hand
        if not MouseMode: 
            MouseMode = True
            print '!!! Mouse Mode on !!!'
        else:
            MouseMode = False
            print '!!! Mouse Mode off !!!'
            
    elif k == ord('d'): # Mouse mode to capture a hand
        if not DemoMode: 
            DemoMode = True
            print '!!! Demo Mode on !!!'
        else:
            DemoMode = False
            print '!!! Demo Mode off !!!'
            
    elif GUIMode:
        if detectedClass:
            imgClass = detectedClass
            X = x
            Y = y
            W = w
            H = h
            Img = img
            
        elif detectedClass2:
            imgClass = detectedClass2 
            X = x2
            Y = y2
            W = w2
            H = h2
            Img = img2
            
        if imgClass:
            # get a crop
            crop = getCrop(X, Y, W, H, margin, Img)

            # make a crop a 200x200 pixel
            fileName = saveCrop(crop, imgClass)

            # save into csv file
            saveCsv(fileName, imgClass)

            # restore the initial state
            imgClass = detectedClass = detectedClass2 = None
            
    elif DemoMode:
        X = x
        Y = y
        W = w
        H = h
        Img = img
        crop = getCrop(X, Y, W, H, margin, Img)
        predictedClass = ML.predict(crop)
        
        if predictedClass != prevClass:
            prevClass = predictedClass
            print predictedClass
            
        
camera.release()
cv2.destroyAllWindows()


!!! Demo Mode on !!!
probability=0.517108, class= lq
probability=0.185994, class= rq
probability=0.074479, class= mw
probability=0.056596, class=vq
probability=0.042515, class= re
 lq
probability=0.409134, class=vc
probability=0.364801, class=vq
probability=0.106212, class=v3
probability=0.078056, class= rc
probability=0.023243, class= lq
vc
probability=0.391190, class=vq
probability=0.387972, class=vc
probability=0.103243, class=v3
probability=0.077422, class= rc
probability=0.022013, class= lq
vq
probability=0.383733, class=vc
probability=0.358545, class=vq
probability=0.115050, class=v3
probability=0.085994, class= rc
probability=0.035301, class= lq
vc
probability=0.519096, class=vq
probability=0.324698, class=vc
probability=0.074669, class=v3
probability=0.039319, class= rc
probability=0.027184, class= lq
vq
probability=0.703211, class=vq
probability=0.153631, class=vc
probability=0.061484, class=v3
probability=0.048710, class= lq
probability=0.013673, class= rc
probability=0.82869

probability=0.587720, class=ve
probability=0.150432, class=v1
probability=0.110824, class=v3
probability=0.028483, class=vo
probability=0.017611, class= lo
ve
probability=0.640734, class=ve
probability=0.150343, class=v1
probability=0.097338, class=vo
probability=0.031769, class=v3
probability=0.022356, class=v4
probability=0.846789, class=ve
probability=0.041930, class=v1
probability=0.020224, class=v3
probability=0.013918, class=vq
probability=0.012746, class=vo
probability=0.840756, class=ve
probability=0.032899, class=v1
probability=0.027353, class=vq
probability=0.022393, class=v3
probability=0.012423, class= lq
probability=0.843222, class=ve
probability=0.031162, class=v1
probability=0.029642, class=vq
probability=0.023056, class=v3
probability=0.011690, class= lq
probability=0.838765, class=ve
probability=0.039706, class=vq
probability=0.024461, class=v1
probability=0.023274, class=v3
probability=0.012109, class=vw
probability=0.824933, class=ve
probability=0.054608, class=vq
pr

probability=0.919424, class=ve
probability=0.038476, class=v1
probability=0.010295, class=vo
probability=0.008125, class=vw
probability=0.004763, class=vq
probability=0.932995, class=ve
probability=0.031136, class=v1
probability=0.007647, class=vw
probability=0.007010, class=vo
probability=0.005165, class=vq
probability=0.919750, class=ve
probability=0.035365, class=v1
probability=0.009221, class=vo
probability=0.008912, class=vw
probability=0.006127, class=vq
probability=0.926237, class=ve
probability=0.030988, class=v1
probability=0.008553, class=vw
probability=0.008390, class=vo
probability=0.005977, class=vq
probability=0.927184, class=ve
probability=0.032266, class=v1
probability=0.008759, class=vo
probability=0.007833, class=vw
probability=0.005566, class=vq
probability=0.927843, class=ve
probability=0.025873, class=v1
probability=0.009464, class=vw
probability=0.007711, class=vo
probability=0.007353, class=vq
probability=0.926048, class=ve
probability=0.026762, class=v1
probabil

probability=0.931517, class=ve
probability=0.025444, class=vo
probability=0.016643, class=v1
probability=0.009830, class=v4
probability=0.006190, class=v3
probability=0.941600, class=ve
probability=0.020594, class=vo
probability=0.015323, class=v1
probability=0.007494, class=v4
probability=0.005207, class=v3
probability=0.945853, class=ve
probability=0.018671, class=vo
probability=0.014954, class=v1
probability=0.007051, class=v4
probability=0.004973, class=v3
probability=0.926083, class=ve
probability=0.027399, class=vo
probability=0.021306, class=v1
probability=0.009324, class=v4
probability=0.006328, class=v3
probability=0.929343, class=ve
probability=0.027922, class=vo
probability=0.019498, class=v1
probability=0.009054, class=v4
probability=0.005126, class=v3
probability=0.934854, class=ve
probability=0.024331, class=vo
probability=0.014715, class=v1
probability=0.009058, class=v4
probability=0.005978, class=v3
probability=0.922129, class=ve
probability=0.031418, class=vo
probabil

probability=0.934134, class=ve
probability=0.026763, class=vo
probability=0.015986, class=v1
probability=0.010097, class=v4
probability=0.007975, class=v3
probability=0.948973, class=ve
probability=0.018662, class=vo
probability=0.009895, class=v1
probability=0.008569, class=v4
probability=0.007765, class=v3
probability=0.928823, class=ve
probability=0.030246, class=vo
probability=0.015371, class=v1
probability=0.010869, class=v4
probability=0.009445, class=v3
probability=0.941459, class=ve
probability=0.022818, class=vo
probability=0.011326, class=v1
probability=0.009643, class=v4
probability=0.009306, class=v3
probability=0.945590, class=ve
probability=0.022427, class=vo
probability=0.010707, class=v1
probability=0.008357, class=v3
probability=0.007503, class=v4
probability=0.947888, class=ve
probability=0.020896, class=vo
probability=0.010879, class=v1
probability=0.007657, class=v3
probability=0.007251, class=v4
probability=0.946376, class=ve
probability=0.021548, class=vo
probabil

probability=0.901007, class=ve
probability=0.035104, class=vo
probability=0.018443, class=v1
probability=0.017630, class=v3
probability=0.017418, class=v4
probability=0.903617, class=ve
probability=0.035753, class=vo
probability=0.017183, class=v3
probability=0.016601, class=v1
probability=0.016298, class=v4
probability=0.893713, class=ve
probability=0.043661, class=vo
probability=0.018998, class=v1
probability=0.018295, class=v4
probability=0.015451, class=v3
probability=0.905045, class=ve
probability=0.032276, class=vo
probability=0.018915, class=v3
probability=0.017210, class=v4
probability=0.016039, class=v1
probability=0.905436, class=ve
probability=0.034285, class=vo
probability=0.017140, class=v1
probability=0.016732, class=v4
probability=0.016215, class=v3
probability=0.908374, class=ve
probability=0.034867, class=vo
probability=0.017536, class=v1
probability=0.014333, class=v4
probability=0.012861, class=v3
probability=0.890097, class=ve
probability=0.045458, class=vo
probabil